In [ ]:
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from scipy.signal import medfilt
import pickle as pk

import numpy.random as nr
import matplotlib.pyplot as plt
import sklearn
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn import preprocessing
import sklearn.model_selection as ms
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score

In [ ]:
cd /content/drive/MyDrive/FORCE-Lithology-Prediction

/content/drive/MyDrive/FORCE-Lithology-Prediction


In [ ]:
A = np.load('/content/drive/My Drive/FORCE-Lithology-Prediction/penalty_matrix.npy')
 
def score(y_true, y_pred):
    S = 0.0
    y_true = y_true.astype(int)
    y_pred = y_pred.astype(int)
    for i in range(0, y_true.shape[0]):
        S -= A[y_true[i], y_pred[i]]
    return S/y_true.shape[0]
 
def evaluate(model):
    feat_imp = pd.Series(model.feature_importances_).sort_values(ascending=False)
    plt.figure(figsize=(24,8))
    feat_imp.plot(kind='bar', title=f'Feature Importances {len(model.feature_importances_)}')
    plt.ylabel('Feature Importance Score')

def show_evaluation(pred, true):
  print(f'Default score: {score(true.values, pred)}')
  print(f'Accuracy is: {accuracy_score(true, pred)}')
  print(f'F1 is: {f1_score(pred, true.values, average="weighted")}')

In [ ]:
#importing miles
nr.seed(42)
train = pd.read_csv('/content/drive/My Drive/FORCE-Lithology-Prediction/train1.csv')
test = pd.read_csv('/content/drive/My Drive/FORCE-Lithology-Prediction/Test.csv', sep=';')
valid1 = pd.read_csv('/content/drive/My Drive/FORCE-Lithology-Prediction/valid4.csv')
valid2 = pd.read_csv('/content/drive/My Drive/FORCE-Lithology-Prediction/valid5.csv')
valid3 = pd.concat((valid1, valid2)).reset_index(drop=True)

In [ ]:
def Normalize_Data_by_Well(dataFrame,col='GR'):
    wells = dataFrame['WELL'].unique()
    values = []
    for well in wells:
        min_value = dataFrame[dataFrame['WELL'] == well][col].min()
        max_value = dataFrame[dataFrame['WELL'] == well][col].max()
        col_normalized = (dataFrame[dataFrame['WELL'] == well][col].values-min_value)/(max_value-min_value)
        values = values + list(col_normalized)
    return values

def Delta_Feature(dataFrame,col='GR',inverted=False):
    wells = dataFrame['WELL'].unique()
    values = []
    for well in wells:
        col_values = dataFrame[dataFrame['WELL'] == well][col].values
        col_values_ = np.array([col_values[0]]+list(col_values[:-1]))
        delta_col_values = col_values-col_values_
        if inverted:
            delta_col_values=-delta_col_values
        values = values + list(delta_col_values)
    return values


def Add_New_Features(dataFrame):
    data = dataFrame['RHOB'].values  
    data = ((154.497/data) - 57.261)
    dataFrame['Carbon_Index'] = data
    dataFrame['Normalized_RHOB'] = Normalize_Data_by_Well(dataFrame,col='RHOB')
    dataFrame['Normalized_GR'] = Normalize_Data_by_Well(dataFrame,col='GR')    
    dataFrame['Delta_DTC'] = Delta_Feature(dataFrame,col='DTC',inverted=True)
    dataFrame['Delta_RHOB'] = Delta_Feature(dataFrame,col='RHOB',inverted=True)    
    dataFrame['Delta_GR'] = Delta_Feature(dataFrame,col='GR',inverted=True)
    dataFrame['Delta_DEPTH_MD'] = Delta_Feature(dataFrame,col='DEPTH_MD')
    dataFrame['Delta_Carbon_Index'] = Delta_Feature(dataFrame,col='Carbon_Index')
    
    return dataFrame

def Fill_Data(dataFrame,fill_formation,fill_BS,fill_with_median):
    dataFrame.FORMATION = dataFrame.FORMATION.fillna(fill_formation)
    dataFrame.BS = dataFrame.BS.fillna(fill_BS)
    dataFrame.fillna(fill_with_median, inplace=True)
    #dataFrame = pd.get_dummies(dataFrame, columns=['FORMATION'], drop_first=True)
    dataFrame['FORMATION_encoded'] = dataFrame['FORMATION'].astype('category')
    dataFrame['FORMATION_encoded'] = dataFrame['FORMATION_encoded'].cat.codes
    dataFrame.drop('FORMATION', inplace=True, axis=1)

    return dataFrame

In [ ]:
train.shape, valid1.shape, valid2.shape

((964965, 29), (114079, 29), (91467, 29))

In [ ]:
ntrain = train.shape[0]
ntest = test.shape[0]
nvalid1 = valid1.shape[0]
nvalid2 = valid2.shape[0]
nvalid3 = valid3.shape[0]
target = train.FORCE_2020_LITHOFACIES_LITHOLOGY.copy()
df = pd.concat((train, test, valid1, valid2, valid3)).reset_index(drop=True)

In [ ]:
df.shape

(1512843, 29)

In [ ]:
df = Add_New_Features(df)

df = df[['DEPTH_MD', 'FORMATION', 'CALI', 'RSHA', 'RMED', 'RDEP', 'RHOB','GR',
         'NPHI', 'PEF', 'DTC', 'SP', 'BS', 'DRHO','FORCE_2020_LITHOFACIES_LITHOLOGY',
         'Carbon_Index','Delta_Carbon_Index','Delta_DTC','Delta_RHOB','Delta_DEPTH_MD',
         'Delta_GR','Normalized_GR','Normalized_RHOB','X_LOC','Y_LOC']]

fill_formation = df.FORMATION.value_counts().index[0]
fill_BS = 12.250001
fill_with_median = df.median()

df = Fill_Data(df,fill_formation,fill_BS,fill_with_median)

output = "FORCE_2020_LITHOFACIES_LITHOLOGY"

X = df.drop([output], axis=1)
y = df[output]

lithology_numbers = {30000: 0,
                 65030: 1,
                 65000: 2,
                 80000: 3,
                 74000: 4,
                 70000: 5,
                 70032: 6,
                 88000: 7,
                 86000: 8,
                 99000: 9,
                 90000: 10,
                 93000: 11}
y = y.map(lithology_numbers)

In [ ]:
lithology = train['FORCE_2020_LITHOFACIES_LITHOLOGY']
valid1_lithology = valid1['FORCE_2020_LITHOFACIES_LITHOLOGY']
valid2_lithology = valid2['FORCE_2020_LITHOFACIES_LITHOLOGY']
valid3_lithology = valid3['FORCE_2020_LITHOFACIES_LITHOLOGY']
 
lithology_numbers = {30000: 0,
                 65030: 1,
                 65000: 2,
                 80000: 3,
                 74000: 4,
                 70000: 5,
                 70032: 6,
                 88000: 7,
                 86000: 8,
                 99000: 9,
                 90000: 10,
                 93000: 11}
 
lithology = lithology.map(lithology_numbers)
valid1_lithology = valid1_lithology.map(lithology_numbers)
valid2_lithology = valid2_lithology.map(lithology_numbers)
valid3_lithology = valid3_lithology.map(lithology_numbers)

In [ ]:
data = df.copy()
 
train2 = data[:ntrain].copy()
target = train2.FORCE_2020_LITHOFACIES_LITHOLOGY.copy()
validation1_target = valid1.FORCE_2020_LITHOFACIES_LITHOLOGY.copy()
validation2_target = valid2.FORCE_2020_LITHOFACIES_LITHOLOGY.copy()
train2.drop(['FORCE_2020_LITHOFACIES_LITHOLOGY'], axis=1, inplace=True)
 
test2 = data[ntrain:(ntest+ntrain)].copy()
test2.drop(['FORCE_2020_LITHOFACIES_LITHOLOGY'], axis=1, inplace=True)
test2 = test2.reset_index(drop=True)
 
validation1 = data[(ntest+ntrain):(ntest+ntrain+nvalid1)].copy()
validation1.drop(['FORCE_2020_LITHOFACIES_LITHOLOGY'], axis=1, inplace=True)
validation1 = validation1.reset_index(drop=True)
 
validation2 = data[(ntrain+ntest+nvalid1): (ntrain+ntest+nvalid1+nvalid2)].copy()
validation2.drop(['FORCE_2020_LITHOFACIES_LITHOLOGY'], axis=1, inplace=True)
validation2 = validation2.reset_index(drop=True)
 
 
validation3 = data[(ntrain+ntest+nvalid1+nvalid2):].copy()
validation3.drop(['FORCE_2020_LITHOFACIES_LITHOLOGY'], axis=1, inplace=True)
validation3 = validation3.reset_index(drop=True)
 
print(train2.shape, test2.shape, validation1.shape, valid1.shape, validation2.shape, validation3.shape, valid2.shape)

(964965, 24) (136786, 24) (114079, 24) (114079, 29) (91467, 24) (205546, 24) (91467, 29)


In [ ]:
traindata = train2
testdata = test2

In [ ]:
class Model():
    
    def __init__(self, train, validation1, validation2, validation3, train_label, valid_label1, valid_label2, valid_label3):
        
        
        self.train = train
        #self.test = test
        self.validation1 = validation1
        self.validation2 = validation2
        self.validation3 = validation3
        self.train_label = train_label
        self.valid_label1 = valid_label1
        self.valid_label2 = valid_label2
        self.valid_label3 = valid_label3
        
    def __call__(self, plot = True):
        return self.fit(plot)
    
    def fit(self, plot):
 
      #self.x_train, self.x_test, self.y_train, self.y_test = ms.train_test_split(self.train, 
                                                                                  # pd.DataFrame(np.array(self.train_label)), 
                                                                                   #test_size=0.2,
                                                                                   #random_state=212)
      #self.x_train = self.train.iloc[:700000]
      #self.x_test = self.train.iloc[700000:]
      #self.y_train = pd.DataFrame(self.train_label).iloc[:700000]
      #self.y_test = pd.DataFrame(self.train_label).iloc[700000:]
      
      def show_evaluation(pred, true):
        
        print(f'Default score: {score(true.values, pred)}')
        print(f'Accuracy is: {accuracy_score(true, pred)}')
        print(f'F1 is: {f1_score(pred, true.values, average="weighted")}')
 
      split = 10
      kf = StratifiedKFold(n_splits=split, random_state=42, shuffle=True)
  
      #pred = np.zeros((len(self.test), 12))
      val1 = np.zeros((len(self.validation1), 12))
      val2 = np.zeros((len(self.validation2), 12))
      val3 = np.zeros((len(self.validation3), 12))
 
      model = XGBClassifier(n_estimators=100, max_depth=5, booster='gbtree',
                            objective='multi:softprob', learning_rate=0.1, random_state=0,
                            subsample=0.9, colsample_bytree=0.9, tree_method='gpu_hist',
                            eval_metric='mlogloss', verbose=2020, reg_lambda=300)

      #model = RandomForestClassifier(n_estimators=100, random_state= 42, min_samples_split=13,
                                     #class_weight='balanced', n_jobs=-1,criterion = 'entropy', verbose=2)
      
      i = 1
      for (train_index, test_index) in kf.split(pd.DataFrame(traindata), pd.DataFrame(lithology)):
        X_train,X_test = pd.DataFrame(traindata).iloc[train_index], pd.DataFrame(traindata).iloc[test_index]
        Y_train,Y_test = pd.DataFrame(lithology).iloc[train_index],pd.DataFrame(lithology).iloc[test_index]
    
        
        model.fit(X_train, Y_train, early_stopping_rounds=100, eval_set=[(X_test, Y_test)], verbose=100)
        #model.fit(X_train, Y_train)
        prediction1 = model.predict(self.validation1)
        prediction = model.predict(self.validation2)
        print(show_evaluation(prediction1, self.valid_label1))
        print(show_evaluation(prediction, self.valid_label2))
 
        print(f'-----------------------FOLD {i}---------------------')
        i+=1
 
        #pred += model.predict_proba(self.test)
        val1 += model.predict_proba(self.validation1)
        val2 += model.predict_proba(self.validation2)
        val3 += model.predict_proba(self.validation3)
      
      #pred = pd.DataFrame(pred/split)
      val1 = pd.DataFrame(val1/split)
      val2 = pd.DataFrame(val2/split)
      val3 = pd.DataFrame(val3/split)
    
      #pred = np.array(pd.DataFrame(pred).idxmax(axis=1))
      val1 = np.array(pd.DataFrame(val1).idxmax(axis=1))
      val2 = np.array(pd.DataFrame(val2).idxmax(axis=1))
      val3 = np.array(pd.DataFrame(val3).idxmax(axis=1))
 
      print('---------------CROSS VALIDATION COMPLETE')
      print('----------------TEST EVALUATION------------------')
 
      print('----------Valid 1-------------')
      print(show_evaluation(val1, self.valid_label1))
      print('----------Valid 2-------------')
      print(show_evaluation(val2, self.valid_label2))
      print('----------Valid 3-------------')
      print(show_evaluation(val3, self.valid_label3))
                  
      if plot: self.plot_feat_imp(model)
      return val1, val2, val3, model
              
              
    def plot_feat_imp(self, model):
        feat_imp = pd.Series(model.feature_importances_).sort_values(ascending=False)
        plt.figure(figsize=(12,8))
        feat_imp.plot(kind='bar', title='Feature Importances')
        plt.ylabel('Feature Importance Score')

In [ ]:
func1_ = Model(traindata, validation1, validation2, validation3, lithology, valid1_lithology, valid2_lithology, valid3_lithology)
val1, open_test1, open_test11, model1 = func1_()

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[0]	validation_0-mlogloss:2.15447
Will train until validation_0-mlogloss hasn't improved in 100 rounds.
[99]	validation_0-mlogloss:0.424193
Default score: -0.6611295681063123
Accuracy is: 0.7515230673480658
F1 is: 0.783463629676568
None
Default score: -0.5949044464123673
Accuracy is: 0.7784884165874031
F1 is: 0.8096582298725373
None
-----------------------FOLD 1---------------------
[0]	validation_0-mlogloss:2.15404
Will train until validation_0-mlogloss hasn't improved in 100 rounds.
[99]	validation_0-mlogloss:0.421079
Default score: -0.6568551179445823
Accuracy is: 0.7535129164876971
F1 is: 0.7865210309590743
None
Default score: -0.5950096756207157
Accuracy is: 0.779188122492265
F1 is: 0.8103715957300418
None
-----------------------FOLD 2---------------------
[0]	validation_0-mlogloss:2.1536
Will train until validation_0-mlogloss hasn't improved in 100 rounds.


KeyboardInterrupt: ignored

In [ ]:
print(show_evaluation(val1, valid1_lithology))
print(show_evaluation(open_test1, valid2_lithology))
print(show_evaluation(open_test11, valid3_lithology))

Default score: -0.6827439756659859
Accuracy is: 0.7440019635515739
F1 is: 0.7994350729294114
None
Default score: -0.5664665945094952
Accuracy is: 0.7926574611608558
F1 is: 0.8390232361458554
None
Default score: -0.6761053486810739
Accuracy is: 0.7501727107314178
F1 is: 0.8034899884348551
None
